In [28]:
import dspy
import os
from dotenv import load_dotenv
import json
from openai import OpenAI

In [35]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
turbo = dspy.OpenAI(model='gpt-4', api_key=OPENAI_API_KEY)
dspy.configure(lm=turbo)
client = OpenAI()

In [13]:
class Question_Three_Replies(dspy.Signature):
    """Given a tweet with a question, generate three possible unique replies"""

    question = dspy.InputField()
    reply1 = dspy.OutputField(desc="1-5 words")
    reply2 = dspy.OutputField(desc="1-5 words")
    reply3 = dspy.OutputField(desc="1-5 words")
q_3 = dspy.Predict(Question_Three_Replies)
answer = q_3(question="What is the meaning of life?")

In [135]:
def filter_image(image_url, new_filter):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
        messages=[
            {
            "role": "assistant",
            "content": [
                {"type": "text", "text": f"Briefly describe the contents of the image."},
                {
                "type": "image_url",
                "image_url": {
                    "url": image_url,
                },
                },
            ],
            }
        ],
        max_tokens=300,
    )
    prompt = response.dict()["choices"][0]["message"]["content"]
    filtered_prompt = f"Reimagine the following prompt if it were filtered like {new_filter}: {prompt}"
    response = client.images.generate(
        model="dall-e-3",
        prompt=filtered_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    return response

url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
new_filter = "Cyberpunk"
res = filter_image(url, new_filter)
res.dict()["data"][0]["url"]